In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
"""
Loading Pandas to validate some checks in sample data, to optimize time.
"""

from divvy_bike_share_data_analysis.data_loader import load_dataset_to_local_fs
from divvy_bike_share_data_analysis.utils_pyspark import load_data


import glob

# Collecting data to local directory
DATA_COLLECTION_DIR: str = '../data_collection/'

# This is commented out, as I only collect the csvs in the very first run.
if not glob.glob(os.path.join(DATA_COLLECTION_DIR, '*.csv')):
    print('collecting data to local fs')
    # load_dataset_to_local_fs(DATA_COLLECTION_DIR, [2020, 2021, 2022, 2023])

# creating pyspark session and pre-processing data
# divvy_df = utils_pyspark.load_data(DATA_COLLECTION_DIR)

collecting data to local fs
